In [31]:
import tkinter as tk
from tkinter import filedialog

In [34]:
class SampleApp(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        
        self.title('Give me Osu Songs')              # 창 제목 설정
        self.geometry("500x300+250+700")   # 창 크기 (너비x높이+x좌표+y좌표)
        self.resizable(False, False)   # 창 크기 조절 여부        
        
        self._frame = None
        self.switch_frame(StartPage)

    def switch_frame(self, frame_class):
        new_frame = frame_class(self)
        if self._frame is not None:
            self._frame.destroy()
        self._frame = new_frame
        self._frame.pack()


class StartPage(tk.Frame):  # 암기모드'


    def __init__(self, master):  
        tk.Frame.__init__(self, master)
        tk.Frame.configure(self, width = 500, height = 300)

        self.instruction = tk.Label(self, text = "안내: Osu 폴더에 있는 Songs 폴더의 경로를 복사&붙여넣기 해주세요") 
        self.instruction.place(x=25, y=15)

        self.songs_location_show = tk.Label(self, text = "Song Location :") 
        self.songs_location_show.place(x=25, y=70)

        self.extract_location_show = tk.Label(self, text = "Extract Location :")
        self.extract_location_show.place(x=25, y=120)

        self.songs_location_entry = tk.Entry(self)
        self.songs_location_entry.bind("<Return>", lambda event:self.Put_percent())
        self.songs_location_entry.place(x=130, y=70, width=310) 

        self.extract_location_entry = tk.Entry(self)
        self.extract_location_entry.bind("<Return>", lambda event:self.Put_percent())
        self.extract_location_entry.place(x=130, y=120, width=310) 

        self.songs_location = tk.Button(self, text = 'Songs 경로 선택', anchor = 'center')#, command = self.Save_Position)
        self.songs_location.place(x=25, y=50)    



if __name__ == "__main__":
    app = SampleApp()
    app.mainloop()